In [1]:
# ========================== CONFIG ==========================
import os
from datetime import date, timedelta, datetime
from zoneinfo import ZoneInfo

# ---- Date window (start anywhere, include N days)
START_DATE   = date(2025, 10, 28)
NUM_DAYS     = 4
DAY_DATES    = [START_DATE + timedelta(days=i) for i in range(NUM_DAYS)]
# Column labels (change to "%a %m/%d" if you prefer dates in headers)

def _col_label(d: date) -> str:
    try:
        return d.strftime("%A %-m/%-d")     # Mac/Linux
    except ValueError:
        return d.strftime("%A %#m/%#d")     # Windows

COL_LABELS = [_col_label(d) for d in DAY_DATES]


# ---- Output
OUTPUT_DIR   = "output"
OUTPUT_XLSX  = f"__BLOCK 1 StoryPointEL_Listings_{START_DATE}_{NUM_DAYS}.xlsx"
os.makedirs(OUTPUT_DIR, exist_ok=True)



# ---- Channel-number profile (sports-relevant only)
CHANNEL_MAPS = {
    "StoryPoint EL": {
        # Locals
        "CBS": 3, "NBC": 4, "FOX": 6, "ABC": 7,
        # Sports tier
        "ESPN": 11, "ESPNews": 12, "ESPNU": 13, "ESPN2": 14,
        "FS1": 15, "Big Ten": 47, 
        #### FILTERING OUT Golf Channel for now because fetching from ESPN API for golf events is different than other sports
        # "Golf": 52,
        # Entertainment nets that carry sports
        "USA": 18, "TNT": 19, "truTV": 20, "TBS": 21,
    }
}
ACTIVE_CHANNEL_MAP_NAME = "StoryPoint EL"

# ================== SCOREBOARD CATALOG (comment to disable) ==================
SPORTS = [
    ("NFL", ["football/nfl"]),
    ("NCAA FB", ["football/college-football"]),
    ("NBA", ["basketball/nba"]),
    ("NHL", ["hockey/nhl"]),
    ("NCAA Hockey", ["hockey/mens-college-hockey"]),
    # ("NCAA Hockey (M)", ["hockey/mens-college-hockey"]),
    ("MLB", ["baseball/mlb"]),
    
    ("NCAA BB (M)", ["basketball/mens-college-basketball"]),
    ("NCAA BB (W)", ["basketball/womens-college-basketball"]),
    
    ### All Known Available Leagues (uncomment to enable more)
    # ("NCAA Baseball", ["baseball/college-baseball"]),
    # ("NCAA Lacrosse(M)", ["lacrosse/mens-college-lacrosse"]),
    ("NCAA Soccer(M)", ["soccer/usa.ncaa.m.1"]),
    # ("Volleyball(M)", ["volleyball/mens-college-volleyball"]),
    # ("NCAA Water Polo(M)", ["waterpolo/mens-college-water-polo"]),
    ("Volleyball", ["volleyball/womens-college-volleyball"]),
    # ("NCAA Softball", ["softball/college-softball"]),
    # ("NCAA Field Hockey", ["fieldhockey/womens-college-field-hockey"]), # # Field Hockey
    # ("NCAA Ice Hockey(W)", ["hockey/womens-college-hockey"]), # # Womens Ice Hockey
    # ("NCAA Lacrosse(W)", ["lacrosse/womens-college-lacrosse"]), # # Lacrosse (W)
    # ("NCAA Soccer(W)", ["soccer/usa.ncaa.w.1"]), # # Soccer Women
    # ("NCAA Volleyball(W)", ["volleyball/womens-college-volleyball"]), # # Volleyball (W)
    # ("NCAA Water Polo(W)", ["waterpolo/womens-college-water-polo "]), # # Water Polo (W)

    # # Soccer
    ("MLS",   ["soccer/usa.1"]), # MLS USA
    ("EPL",   ["soccer/eng.1"]), # ENGLISH PREMIER
    ("UCL",   ["soccer/uefa.champions"]), # UEFA CHAMPIONS LEAGUE
    # ("La Liga", ["soccer/esp.1"]), # # La Liga
    # ("Bundesliga", ["soccer/ger.1"]), # # Bundesliga
    # ("NWSL",  ["soccer/usa.nwsl"]), # # National Womens NWSL (USA)
    
    # ("Liga MX", ["soccer/mex.1"]), # # Liga MX (Mexico)
]

# Short tags for the grid
SPORT_TAGS = {
    "football/nfl": "NFL",
    "basketball/nba": "NBA",
    "hockey/nhl": "NHL",
    "baseball/mlb": "MLB",
    "football/college-football": "FBS FB",
    "basketball/mens-college-basketball": "(M)CBB",
    "basketball/womens-college-basketball": "(W)CBB",
    "soccer/usa.1": "Soccer - MLS",
    "soccer/eng.1": "Soccer - EPL",
    "soccer/uefa.champions": "Soccer - UCL",
    "golf/pga": "PGA Tour",
    "golf/champions-tour": "Champions Tour",
    "golf/lpga": "LPGA",
    "golf/liv": "LIV Golf",
    "golf/eur": "European Tour",
    "racing/f1": "F1",
    "tennis/atp": "ATP",
    "tennis/wta": "WTA",
}

# ---- Favorites config
# Pro favorites: (team_name_contains, candidate_league_keys)
FAVORITE_PRO_TEAMS = [
    ("Boston Bruins",  ["hockey/nhl"]),
    ("Boston Celtics", ["basketball/nba"]),
]

# Favorite school: name + leagues to scan (add/remove as needed)
FAVORITE_SCHOOL = {
    "name": "Michigan State",
    "leagues": [
        "football/college-football",
        "basketball/mens-college-basketball",
        "basketball/womens-college-basketball",
        "hockey/mens-college-hockey",
        "baseball/college-baseball",
        "softball/college-softball",
        "soccer/mens-college-soccer",
        "soccer/womens-college-soccer",
        "volleyball/womens-college-volleyball",
        # add other college leagues here as you discover ESPN keys you care about
    ],
    # special row styling
    "bg_color": "#18453B",   # forest green
    "font_color": "#FFFFFF", # white
}

# Map ESPN broadcast strings -> canonical channel labels (for the normal rows)
CHANNEL_ALIASES = {
    "abc": "ABC", "abc network": "ABC",
    "fox": "FOX", "fox network": "FOX",
    "cbs": "CBS", "cbs network": "CBS",
    "nbc": "NBC", "nbc network": "NBC", "nbc sports": "NBC", "nbcsn": "NBC",

    "espn": "ESPN", "espn2": "ESPN2", "espnu": "ESPNU",
    "espn news": "ESPNews", "espnnews": "ESPNews",

    "fs1": "FS1", "fox sports 1": "FS1", "fox sports1": "FS1",

    "btn": "Big Ten", "big ten network": "Big Ten",
    "golf channel": "Golf",

    "usa": "USA", "usa network": "USA",
    "tnt": "TNT", "tnt hd": "TNT",
    "tbs": "TBS",
    "trutv": "truTV",
}

## TEST TO EXCLUDE STRERAMING SERVICES
EXCLUDE_STREAMING_KEYS = ("espn+", "espn plus", "espn app", "peacock+", "peacock premium")
LOCAL_TZ = ZoneInfo("America/Detroit")

# ========================== IMPORTS ==========================
import re
import math
import pandas as pd
import requests
from collections import defaultdict

# ========================== HELPERS ==========================
def to_local_timestr(iso_str: str):
    if not iso_str:
        return None, None
    ts_utc = pd.to_datetime(iso_str, errors="coerce", utc=True)
    if pd.isna(ts_utc):
        return None, None
    ts_local = ts_utc.tz_convert(LOCAL_TZ).tz_localize(None)
    try:
        hm = ts_local.strftime("%-I:%M%p").lower()
    except ValueError:
        hm = ts_local.strftime("%#I:%M%p").lower()
    return ts_local, hm

def normalize_key(s: str) -> str:
    return re.sub(r"[^a-z0-9+ ]", "", s.lower()).strip()

def _split_broadcast_tokens(raw: str) -> list[str]:
    s = re.sub(r"[\/&]| and ", ",", raw, flags=re.IGNORECASE)
    parts = [p.strip() for p in s.split(",")]
    return [p for p in parts if p]

def normalize_channel_name(name: str) -> str | None:
    if not name:
        return None
    lk = name.lower()
    if any(x in lk for x in EXCLUDE_STREAMING_KEYS):
        return None
    return CHANNEL_ALIASES.get(normalize_key(name))

def sport_is_womens(league_key: str) -> bool:
    return "womens" in league_key

def team_label(c: dict) -> str:
    t = (c or {}).get("team", {}) or {}
    rank = (c or {}).get("curatedRank", {}).get("current")
    nm = t.get("displayName") or t.get("shortDisplayName") or t.get("name") or ""
    return (f"#{rank} " if rank and rank != 99 else "") + nm

def build_title(comp: dict, prefix_womens=False) -> str:
    comps = comp.get("competitors", []) or []
    by_side = {c.get("homeAway"): c for c in comps}
    away = team_label(by_side.get("away", {}))
    home = team_label(by_side.get("home", {}))
    title = f"{away} at {home}".strip()
    if prefix_womens:
        title = f"(W) {title}"
    return title

def fetch_day_sport_multi(day: date, league_keys: list[str]) -> list[dict]:
    ymd = day.strftime("%Y%m%d")
    last_err = None
    for k in league_keys:
        url = f"https://site.api.espn.com/apis/site/v2/sports/{k}/scoreboard?dates={ymd}"
        try:
            r = requests.get(url, timeout=20)
            r.raise_for_status()
            return r.json().get("events", []) or []
        except Exception as e:
            last_err = e
            continue
    if last_err:
        print(f"[WARN] {day} {'/'.join(league_keys)} fetch failed: {last_err}")
    return []

def extract_broadcast_tokens(ev: dict, comp: dict) -> list[str]:
    candidates = []
    for b in (comp.get("broadcasts") or []):
        media = b.get("media") or {}
        for k in ("shortName", "name"):
            if media.get(k): candidates.append(str(media[k]))
        for k in ("shortName", "name"):
            if b.get(k): candidates.append(str(b[k]))
        for n in (b.get("names") or []):
            candidates.append(str(n))
    for gb in (ev.get("geoBroadcasts") or []):
        chan = gb.get("media", {}).get("shortName") or gb.get("media", {}).get("name")
        if chan: candidates.append(str(chan))
    b = comp.get("broadcast") or {}
    if isinstance(b, dict):
        for k in ("shortName", "name"):
            if b.get(k): candidates.append(str(b[k]))
    tokens, seen = [], set()
    for raw in candidates:
        for tok in _split_broadcast_tokens(raw):
            nk = normalize_key(tok)
            if nk and nk not in seen:
                seen.add(nk)
                tokens.append(tok)
    return tokens

def get_event_tag(league_key: str, sport_label: str) -> str:
    tag = SPORT_TAGS.get(league_key)
    if not tag:
        fallback = {
            "NBA":"NBA","NHL":"NHL","NFL":"NFL","MLB":"MLB","CFB":"CFB",
            "MBB":"MBB","WBB":"WBB","MLS":"MLS","EPL":"EPL","UCL":"UCL",
            "PGA":"PGA","LPGA":"LPGA","LIV":"LIV","DPW":"DPW"
        }.get(sport_label, "SPORT")
        tag = fallback
    return f"({tag})"

def _break_after_at(title: str) -> str:
    return re.sub(r"\s+(at|vs\.?|v\.)\s+", r" \1\n", title, count=1, flags=re.IGNORECASE)

def _comp_has_team(comp: dict, needle: str) -> bool:
    """case-insensitive contains on team display names."""
    if not needle:
        return False
    needle = needle.lower()
    for c in (comp.get("competitors") or []):
        t = (c or {}).get("team", {}) or {}
        for f in ("displayName", "shortDisplayName", "name"):
            val = (t.get(f) or "").lower()
            if val and needle in val:
                return True
    return False

# ========================== GATHER EVENTS ==========================
active_map = CHANNEL_MAPS[ACTIVE_CHANNEL_MAP_NAME]
unknown_broadcasts = defaultdict(int)
rows = []

# Normal channel-based events (filter to active_map)
for day in DAY_DATES:
    for sport_label, league_keys in SPORTS:
        events = fetch_day_sport_multi(day, league_keys)
        if not events:
            continue
        for ev in events:
            for comp in (ev.get("competitions") or []):
                local_dt, hm = to_local_timestr(comp.get("date") or ev.get("date"))
                if local_dt is None or local_dt.date() not in DAY_DATES:
                    continue
                col_label = _col_label(local_dt.date())

                # broadcasts -> channels
                channel_hits = set()
                for raw in extract_broadcast_tokens(ev, comp):
                    canon = normalize_channel_name(raw)
                    if canon:
                        channel_hits.add(canon)
                    else:
                        key = normalize_key(raw)
                        if key and not any(x in key for x in EXCLUDE_STREAMING_KEYS):
                            unknown_broadcasts[raw] += 1

                channel_hits = [c for c in channel_hits if c in active_map]
                if not channel_hits:
                    continue

                title = build_title(comp, prefix_womens=sport_is_womens(league_keys[0]))
                tag_key = next((k for k in league_keys if k in SPORT_TAGS), None)
                tag = f"({SPORT_TAGS.get(tag_key, sport_label)})"

                for ch in channel_hits:
                    rows.append({
                        "col_label": col_label,
                        "date": local_dt.date(),
                        "time_str": hm,
                        "start_dt": local_dt,
                        "sport": sport_label,
                        "league_key": tag_key or league_keys[0],
                        "tag": tag,
                        "channel": ch,
                        "channel_num": active_map.get(ch),
                        "title": title,
                        "fav_row": None,   # normal grid
                    })

# Favorite pro teams (ignore channel filters; force into special rows)
FAV_PRO_ROWS = []  # [(row_num, row_label)]
for i, (team_name, league_keys) in enumerate(FAVORITE_PRO_TEAMS):
    # row_num = -90 + i  # sort before normal channels
    
    row_num = max(active_map.values()) + 10 + i     # place at end of normal channels
    row_label = f"{team_name}"
    FAV_PRO_ROWS.append((row_num, row_label))

    for day in DAY_DATES:
        events = fetch_day_sport_multi(day, league_keys)
        if not events:
            continue
        for ev in events:
            for comp in (ev.get("competitions") or []):
                if not _comp_has_team(comp, team_name):
                    continue
                local_dt, hm = to_local_timestr(comp.get("date") or ev.get("date"))
                if local_dt is None or local_dt.date() not in DAY_DATES:
                    continue
                col_label = _col_label(local_dt.date())
                title = build_title(comp, prefix_womens=sport_is_womens(league_keys[0]))
                tag = get_event_tag(next((k for k in league_keys if k in SPORT_TAGS), league_keys[0]), sport_label=team_name)
                rows.append({
                    "col_label": col_label,
                    "date": local_dt.date(),
                    "time_str": hm,
                    "start_dt": local_dt,
                    "sport": team_name,           # display purpose
                    "league_key": league_keys[0], # best-effort
                    "tag": tag,
                    "channel": row_label,
                    "channel_num": row_num,
                    "title": title,
                    "fav_row": "pro",             # mark as favorite row
                })

# Favorite school (all listed college leagues; special styling)
SCHOOL_ROW_NUM  = 100 # sort after normal channels
SCHOOL_ROW_LABEL= f"MSU \n on B1G+"

for day in DAY_DATES:
    for k in FAVORITE_SCHOOL["leagues"]:
        events = fetch_day_sport_multi(day, [k])
        if not events:
            continue
        for ev in events:
            for comp in (ev.get("competitions") or []):
                if not _comp_has_team(comp, FAVORITE_SCHOOL["name"]):
                    continue
                local_dt, hm = to_local_timestr(comp.get("date") or ev.get("date"))
                if local_dt is None or local_dt.date() not in DAY_DATES:
                    continue
                col_label = _col_label(local_dt.date())
                title = build_title(comp, prefix_womens=sport_is_womens(k))
                tag = f"({SPORT_TAGS.get(k, FAVORITE_SCHOOL['name'])})"
                rows.append({
                    "col_label": col_label,
                    "date": local_dt.date(),
                    "time_str": hm,
                    "start_dt": local_dt,
                    "sport": FAVORITE_SCHOOL["name"],
                    "league_key": k,
                    "tag": tag,
                    "channel": SCHOOL_ROW_LABEL,
                    "channel_num": SCHOOL_ROW_NUM,
                    "title": title,
                    "fav_row": "school",          # special styling
                })

# ---------------- DataFrame ----------------
df = pd.DataFrame(rows)
if df.empty:
    print("No events matched. Check dates, SPORTS, favorites, and mapping.")
else:
    df = df.sort_values(["date","channel_num","start_dt","title"]).reset_index(drop=True)
    df.to_csv(os.path.join(OUTPUT_DIR, "events_flat_storypoint.csv"), index=False)
    print("Wrote flat CSV with", len(df), "rows")

# ========================== EXCEL WRITER (rich text, borders, alt shading + fav rows) ==========================
if not df.empty:
    df["col_label"] = pd.Categorical(df["col_label"], categories=COL_LABELS, ordered=True)

    # Per-cell: (channel_num, channel, col_label) -> [(time, tag, title, fav_row_flag)]
    events_by_cell = defaultdict(list)
    for r in df.itertuples():
        events_by_cell[(r.channel_num, r.channel, r.col_label)].append((r.time_str, r.tag, r.title, r.fav_row))

    # Row order: favorites first, then normal channels by number
    active_map = CHANNEL_MAPS[ACTIVE_CHANNEL_MAP_NAME]
    normal_rows = sorted([(num, ch) for ch, num in active_map.items()], key=lambda x: (x[0], str(x[1])))
    fav_rows = [(SCHOOL_ROW_NUM, SCHOOL_ROW_LABEL)] + FAV_PRO_ROWS
    row_index = fav_rows + normal_rows

    # title/subheader
    def fmt(d: date) -> str: return d.strftime("%B %d, %Y")
    TITLE_TEXT   = f"{fmt(START_DATE)} – {fmt(DAY_DATES[-1])}"
    SUBHEAD_TEXT = f"StoryPoint – East Lansing | Created {datetime.now(LOCAL_TZ).strftime('%B %d, %Y')} by J.Smith"

    # column positions
    first_col     = 1
    num_col       = first_col
    chan_col      = first_col + 1
    first_day_col = chan_col + 1
    last_day_col  = chan_col + len(COL_LABELS)

    out_path = os.path.join(OUTPUT_DIR, OUTPUT_XLSX)
    with pd.ExcelWriter(out_path, engine="xlsxwriter") as xw:
        wb  = xw.book
        ws  = wb.add_worksheet("Week")

        # Per-sport font colors
        SPORT_STYLE = {
            "NFL": "#B22222", "NBA": "#5C2E91", "NHL": "#1F4E79", "MLB": "#0A2463",
            "CFB": "#0B6E4F", "MBB": "#D97706", "WBB": "#C026D3",
            "MLS": "#0B7285", "EPL": "#1D4ED8", "UCL": "#6D28D9", "MCH": "#065F46",
            "PGA": "#047857", "LPGA": "#BE185D", "LIV": "#111827", "DPW": "#B45309", "KFT": "#374151",
        }

        # Base formats
        title_fmt = wb.add_format({"bold": True, "align": "left", "valign": "vcenter", "font_size": 36 })
        sub_fmt   = wb.add_format({"italic": True, "align": "left", "valign": "vcenter", "font_size": 12, "font_color": "#555555"})

        header_fmt = wb.add_format({"bold": True, "align": "center", "valign": "vcenter", "font_size": 14, "border": 1})

        num_fmt         = wb.add_format({"bold": True, "align": "center", "valign": "vcenter", "font_size": 24, "bottom": 1})
        chan_fmt        = wb.add_format({"bold": True, "align": "center",  "valign": "vcenter", "font_size": 22, "bottom": 1})
        num_fmt_shaded  = wb.add_format({"bold": True, "align": "center", "valign": "vcenter", "font_size": 24, "bottom": 1, "bg_color": "#F5F5F5"})
        chan_fmt_shaded = wb.add_format({"bold": True, "align": "center",  "valign": "vcenter", "font_size": 22, "bottom": 1, "bg_color": "#F5F5F5"})

        base_cell_fmt        = wb.add_format({"text_wrap": True, "valign": "vcenter", "bottom": 1})
        base_cell_fmt_shaded = wb.add_format({"text_wrap": True, "valign": "vcenter", "bottom": 1, "bg_color": "#F5F5F5"})

        # Favorite school cell base (forest green bg + white text)
        fav_school_cell_fmt        = wb.add_format({"text_wrap": True, "valign": "vcenter", "bottom": 1, "bg_color": FAVORITE_SCHOOL["bg_color"], "font_color": FAVORITE_SCHOOL["font_color"]})
        # rich text runs for fav school (white)
        fav_school_time_fmt        = wb.add_format({"bold": True, "font_size": 16, "font_color": FAVORITE_SCHOOL["font_color"]})
        fav_school_teams_fmt       = wb.add_format({"font_size": 14, "font_color": FAVORITE_SCHOOL["font_color"]})

        # Titles
        ws.merge_range(0, num_col, 0, last_day_col, f"Live Sports on TV - Week of {fmt(START_DATE)}", title_fmt)
        ws.merge_range(1, num_col, 1, last_day_col, SUBHEAD_TEXT, sub_fmt)

        # Column headers
        ws.write(2, num_col,  "#", num_fmt)
        ws.write(2, chan_col, "Channel", chan_fmt)
        for c, lbl in enumerate(COL_LABELS):
            ws.write(2, first_day_col + c, lbl, header_fmt)

        # Widths
        ws.set_column(num_col,  num_col, 12)
        ws.set_column(chan_col, chan_col, 20)   # wider for "Streaming – …"
        ws.set_column(first_day_col, last_day_col, 36)

        # Helpers
        def _tag_key(tag_text: str) -> str:
            return (tag_text or "").strip().strip("()").upper()

        _format_cache = {}
        def _get_event_formats(tag: str, shaded: bool):
            key = (tag, shaded)
            if key in _format_cache:
                return _format_cache[key]
            color = SPORT_STYLE.get(tag)
            time_kwargs = {"bold": True, "font_size": 16}
            team_kwargs = {"font_size": 14}
            if color:
                time_kwargs["font_color"] = color
                team_kwargs["font_color"] = color
            tt_fmt = wb.add_format(time_kwargs)
            tm_fmt = wb.add_format(team_kwargs)
            cell_fmt = base_cell_fmt_shaded if shaded else base_cell_fmt
            _format_cache[key] = (tt_fmt, tm_fmt, cell_fmt)
            return _format_cache[key]

        def write_rich_cell(row_idx: int, col_idx: int, evts, shaded: bool, fav_kind: str | None):
            """
            evts: list of (time_str, tag_text, title, fav_row_flag)
            fav_kind: None | "pro" | "school"
            """
            if not evts:
                # choose correct base format
                base_fmt = (
                    fav_school_cell_fmt if fav_kind == "school"
                    else (base_cell_fmt_shaded if shaded else base_cell_fmt)
                )
                ws.write(row_idx, col_idx, "", base_fmt)
                return

            parts = []
            for i, (time_str, tag_text, title, _) in enumerate(evts):
                tag = _tag_key(tag_text)
                if fav_kind == "school":
                    # override to white-on-green for school favorites
                    parts.extend([fav_school_time_fmt, f"{time_str}: ({tag})"])
                    parts.append("\n")
                    parts.extend([fav_school_teams_fmt, _break_after_at(title)])
                else:
                    tt_fmt, tm_fmt, _ = _get_event_formats(tag, shaded)
                    parts.extend([tt_fmt, f"{time_str}: ({tag})"])
                    parts.append("\n")
                    parts.extend([tm_fmt, _break_after_at(title)])
                if i != len(evts) - 1:
                    parts.append("\n")

            # trailing format controls background/wrap
            trailing_fmt = (
                fav_school_cell_fmt if fav_kind == "school"
                else (base_cell_fmt_shaded if shaded else base_cell_fmt)
            )
            parts.append(trailing_fmt)
            ws.write_rich_string(row_idx, col_idx, *parts)

        # Body with alternating shading for normal/pro rows; school uses its own bg
        current_row = 3
        for idx, (num, ch) in enumerate(row_index):
            fav_kind = None
            if num == SCHOOL_ROW_NUM and ch == SCHOOL_ROW_LABEL:
                shaded = False     # ignored; school row has its own bg
                fav_kind = "school"
                row_num_fmt  = num_fmt          # no gray stripe
                row_chan_fmt = chan_fmt
            elif num < 0:
                # pro favorite rows: keep alternating stripe
                shaded = (idx % 2 == 1)
                row_num_fmt  = num_fmt_shaded  if shaded else num_fmt
                row_chan_fmt = chan_fmt_shaded if shaded else chan_fmt
                fav_kind = "pro"
            else:
                shaded = (idx % 2 == 1)
                row_num_fmt  = num_fmt_shaded  if shaded else num_fmt
                row_chan_fmt = chan_fmt_shaded if shaded else chan_fmt

            # Row header cells
            try:
                ws.write(current_row, num_col, int(num) if num is not None else "", row_num_fmt)
            except Exception:
                ws.write(current_row, num_col, "", row_num_fmt)
            ws.write(current_row, chan_col, str(ch), row_chan_fmt)

            # Day cells
            for c, lbl in enumerate(COL_LABELS):
                evts = events_by_cell.get((num, ch, lbl), [])
                write_rich_cell(current_row, first_day_col + c, evts, shaded, fav_kind)

            current_row += 1

                # Footer or summary rows could be added here if needed

        # ---- Page setup / print settings ----
        from xlsxwriter.utility import xl_range

        last_row = current_row - 1              # last row we wrote
        first_row = 0                           # include title/subheader
        first_col = num_col                     # start at the "#" column
        last_col  = last_day_col                # last day column

        # Page orientation and paper
        ws.set_landscape()          # Landscape
        ws.set_paper(1)             # 1 = Letter (8.5" x 11"). Use 9 for A4.

        # Margins Set to Max
        
        ws.set_margins(left=0.2, right=0.2, top=0.2, bottom=0.2)

        # Center on page (nice for single sheet handouts)
        ws.center_horizontally()
        ws.center_vertically()

        # Fit to a single printed page (1 page wide × 1 page tall)
        ws.fit_to_pages(1, 1)

        # Define the print area so the sheet opens ready to print one page
        ws.print_area(first_row, first_col, last_row, last_col)

        # Optional: repeat header row (the row with "# / Channel / dates") on each printed page
        # (harmless even when we fit to one page)
        ws.repeat_rows(2, 2)  # zero-based row index; your headers are on row 2

        # Optional: show/hide gridlines in print (2 = hide on screen & print)
        ws.hide_gridlines(2)

        # Optional: header/footer
        ws.set_header('&L&"Calibri,Bold"&12StoryPoint – East Lansing'
                    '&R&"Calibri"&10Printed &D')
        # ws.set_footer('&CPage &P of &N')



    print("Wrote workbook:", out_path)

# ========================== UNKNOWN BROADCASTS LOG ==========================
if unknown_broadcasts:
    pd.DataFrame(
        sorted(unknown_broadcasts.items(), key=lambda x: (-x[1], x[0])),
        columns=["raw_broadcast_string","count"]
    ).to_csv(os.path.join(OUTPUT_DIR, "unknown_broadcasts.csv"), index=False)
    print("Wrote unknown_broadcasts.csv (consider adding aliases).")


[WARN] 2025-10-28 softball/college-softball fetch failed: 400 Client Error: Bad Request for url: https://site.api.espn.com/apis/site/v2/sports/softball/college-softball/scoreboard?dates=20251028
[WARN] 2025-10-28 soccer/mens-college-soccer fetch failed: 400 Client Error: Bad Request for url: https://site.api.espn.com/apis/site/v2/sports/soccer/mens-college-soccer/scoreboard?dates=20251028
[WARN] 2025-10-28 soccer/womens-college-soccer fetch failed: 400 Client Error: Bad Request for url: https://site.api.espn.com/apis/site/v2/sports/soccer/womens-college-soccer/scoreboard?dates=20251028
[WARN] 2025-10-29 softball/college-softball fetch failed: 400 Client Error: Bad Request for url: https://site.api.espn.com/apis/site/v2/sports/softball/college-softball/scoreboard?dates=20251029
[WARN] 2025-10-29 soccer/mens-college-soccer fetch failed: 400 Client Error: Bad Request for url: https://site.api.espn.com/apis/site/v2/sports/soccer/mens-college-soccer/scoreboard?dates=20251029
[WARN] 2025-10-

PermissionError: [Errno 13] Permission denied: 'output\\__BLOCK 1 StoryPointEL_Listings_2025-10-28_4.xlsx'

In [ ]:
break

In [ ]:
# 